In [9]:
import numpy as np
from skyfield.api import EarthSatellite, load, wgs84, utc
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import tkinter as tk
from tkinter import messagebox
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [2]:
# Parameters
NUM_SATELLITES = 5
FORCED_COLLISIONS = [(0, 1, 20), (2, 3, 40)]  # (sat1_idx, sat2_idx, time_idx)
DURATION_MINUTES = 60
COLLISION_THRESHOLD_KM = 9000

# Load TLE data
with open('gp.txt') as file:
    lines = file.readlines()

ts = load.timescale()
start = datetime.now().replace(tzinfo=utc)

# Generate times
times = ts.utc(start.year, start.month, start.day, start.hour,
               start.minute + np.arange(0, DURATION_MINUTES, 1))

# Load N satellites
satellites = []
names = []

for i in range(0, NUM_SATELLITES * 3, 3):
    if i + 2 >= len(lines):
        break  
    name = lines[i].strip()
    tle1 = lines[i+1].strip()
    tle2 = lines[i+2].strip()
    sat = EarthSatellite(tle1, tle2, name, ts)
    satellites.append(sat)
    names.append(name)

# Get positions
# Get positions with noise added
positions = []

# Noise level in km (adjust as needed)
NOISE_LEVEL_KM = 0.2  # small noise up to ±0.2 km

for sat in satellites:
    geo = sat.at(times)
    pos = geo.position.km.T  # (T, 3)

    # Add Gaussian noise to simulate drift/errors
    noise = np.random.normal(loc=0.0, scale=NOISE_LEVEL_KM, size=pos.shape)
    noisy_pos = pos + noise
    positions.append(noisy_pos)

# Force collisions with tiny noise (to simulate very close approach)
for (i, j, t_idx) in FORCED_COLLISIONS:
    if i >= len(positions) or j >= len(positions):
        print(f"⚠️ Invalid satellite index: i={i}, j={j}, skipping...")
        continue
    if t_idx >= len(times):
        print(f"⚠️ Invalid time index: t_idx={t_idx}, skipping...")
        continue

    offset = np.random.uniform(-0.4, 0.4, size=3)
    positions[j][t_idx] = positions[i][t_idx] + offset

# Generate dataset
data = []
actual_satellites = len(positions)
time_steps = len(times)

print(f"Satellites available: {actual_satellites}, Time steps: {time_steps}")
for t in range(time_steps):
    for i in range(actual_satellites*10000):
        for j in range(i+1, actual_satellites):
            p1 = positions[i][t]
            p2 = positions[j][t]
            distance = np.linalg.norm(p1 - p2)
            label = 1 if distance <= COLLISION_THRESHOLD_KM else 0
            features = np.concatenate((p1, p2, [distance]))
            data.append(np.append(features, label))

df = pd.DataFrame(data, columns=['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'distance', 'label'])
df.to_csv('generated_collision_dataset.csv', index=False)

print("✅ Generated dataset with forced collisions saved as 'generated_collision_dataset.csv'")

Satellites available: 5, Time steps: 60
✅ Generated dataset with forced collisions saved as 'generated_collision_dataset.csv'


In [3]:
# Load dataset
df = pd.read_csv('generated_collision_dataset.csv')
df
# Features and label
X = df[['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'distance']]
y = df['label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
print("\n🔍 Classification Report:")
print(classification_report(y_test, y_pred))

print("\n🟦 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save model
joblib.dump(model, 'collision_predictor_model.pkl')
print("\n✅ Model saved as 'collision_predictor_model.pkl'")


🔍 Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        78
         1.0       1.00      1.00      1.00        42

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120


🟦 Confusion Matrix:
[[78  0]
 [ 0 42]]

✅ Model saved as 'collision_predictor_model.pkl'


In [4]:
# Load the trained model
model = joblib.load('collision_predictor_model.pkl')

# GUI App
app = tk.Tk()
app.title("Satellite Collision Predictor")
app.geometry("600x700")
app.configure(bg="#f0f0f0")

# Title Label
title_label = tk.Label(app, text="Satellite Collision Prediction", font=("Arial", 16, "bold"), bg="#f0f0f0")
title_label.pack(pady=10)

# Input Frame
input_frame = tk.Frame(app, bg="#f0f0f0")
input_frame.pack(pady=10)

# Input fields
labels = ['x1', 'y1', 'z1', 'x2', 'y2', 'z2']
entries = {}
for i, label_text in enumerate(labels):
    label = tk.Label(input_frame, text=f"{label_text} (km):", font=("Arial", 12), bg="#f0f0f0")
    label.grid(row=i, column=0, padx=10, pady=5, sticky='e')
    entry = tk.Entry(input_frame, width=20)
    entry.grid(row=i, column=1, pady=5)
    entries[label_text] = entry

# Visualization Frame
viz_frame = tk.Frame(app, bg="#f0f0f0")
viz_frame.pack(pady=10, fill="both", expand=True)

# Clear old plot
def clear_plot():
    for widget in viz_frame.winfo_children():
        widget.destroy()

# Embedded Visualization function
def visualize(p1, p2):
    clear_plot()
    fig = plt.Figure(figsize=(5, 4), dpi=100)
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(*p1, color='blue', s=50, label='Satellite 1')
    ax.scatter(*p2, color='red', s=50, label='Satellite 2')
    ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], color='green', linestyle='--', label='Distance Vector')

    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Z (km)')
    ax.set_title('Satellite Positions')
    ax.legend()

    canvas = FigureCanvasTkAgg(fig, master=viz_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill="both", expand=True)

# Predict Button
def predict_collision():
    try:
        # Get inputs
        x1 = float(entries['x1'].get())
        y1 = float(entries['y1'].get())
        z1 = float(entries['z1'].get())
        x2 = float(entries['x2'].get())
        y2 = float(entries['y2'].get())
        z2 = float(entries['z2'].get())

        # Compute distance
        p1 = np.array([x1, y1, z1])
        p2 = np.array([x2, y2, z2])
        distance = np.linalg.norm(p1 - p2)

        # Predict using model
        features = np.concatenate((p1, p2, [distance])).reshape(1, -1)
        risk = model.predict(features)[0]

        # Visualize
        visualize(p1, p2)

        # Show result
        if risk == 1:
            messagebox.showwarning("⚠️ Collision Risk", f"Collision Risk Detected!\nDistance: {distance:.3f} km")
        else:
            messagebox.showinfo("✅ Safe", f"No Collision Risk.\nDistance: {distance:.3f} km")

    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numbers for all fields.")

# Predict Button UI
predict_btn = tk.Button(app, text="Predict Collision", font=("Arial", 14), bg="#4CAF50", fg="white",
                        command=predict_collision)
predict_btn.pack(pady=10)

# Run app
app.mainloop()


C:\Users\swaya\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
